# Image Operations
___

Almost all the operations in this section is mainly related to Numpy rather than OpenCV. A good knowledge
of Numpy is required to write better optimized code with OpenCV.

## Accessing and Modifying pixel values
Let's load a color image first:

In [20]:
import cv2
import numpy as np
from matplotlib.pyplot import *
%matplotlib inline

img = cv2.imread('images/googlelogo.jpg')

You can access a pixel value by its row and column coordinates.
For BGR image, it returns an array of Blue, Green, Red values.
For grayscale image, just corresponding intensity is returned.


In [2]:
px = img[150,300]
print px

[ 57  65 236]


In [3]:
# accessing only blue pixel
blue = img[100,100,0]
print blue

255


You can modify the pixel values the same way.


In [4]:
img[100,100] = [255,255,255]
print img[100,100]

[255 255 255]


## Better pixel accessing and editing method :

In [5]:
# accessing RED value
img.item(10,10,2)

255

In [6]:
# modifying RED value
img.itemset((10,10,2),100)
img.item(10,10,2)

100

## Accessing Image Properties
Image properties include number of rows, columns and channels, type of image data, number of pixels etc.
Shape of image is accessed by img.shape. It returns a tuple of number of rows, columns and channels (if image is
color):

In [7]:
print img.shape

(500L, 500L, 3L)


If image is grayscale, tuple returned contains only number of rows and columns. So it is a good method to
check if loaded image is grayscale or color image.

Total number of pixels is accessed by img.size:

In [8]:
print img.size

750000


Image datatype is obtained by img.dtype:


In [9]:
print img.dtype

uint8


## Image ROI
Sometimes, you will have to play with certain region of images. For eye detection in images, first perform face
detection over the image until the face is found, then search within the face region for eyes. This approach improves
accuracy (because eyes are always on faces :D ) and performance (because we search for a small area).

ROI is again obtained using Numpy indexing.

In [21]:
region = img[250:350, 400:220]
img[100:200, 250:70] = region

## Splitting and Merging Image Channels
The B,G,R channels of an image can be split into their individual planes when needed. Then, the individual channels
can be merged back together to form a BGR image again. This can be performed by:


In [22]:
b,g,r = cv2.split(img)
img = cv2.merge((b,g,r))
#Or
b = img[:,:,0]


Suppose, you want to make all the red pixels to zero, you need not split like this and put it equal to zero. You can
simply use Numpy indexing which is faster.

In [24]:
img[:,:,2] = 0

## Making Borders for Images (Padding)
If you want to create a border around the image, something like a photo frame, you can use _cv2.copyMakeBorder()_
function. But it has more applications for convolution operation, zero padding etc. This function takes following
arguments:
* src - input image
* top, bottom, left, right - border width in number of pixels in corresponding directions
* borderType - Flag defining what kind of border to be added. It can be following types:
    * cv2.BORDER_CONSTANT - Adds a constant colored border. The value should be given as next argument.
    * cv2.BORDER_REFLECT - Border will be mirror reflection of the border elements, like this : fedcba| abcdefgh|hgfedcb
    * cv2.BORDER_REFLECT_101 or cv2.BORDER_DEFAULT - Same as above, but with a slight change, like this : gfedcb|abcdefgh|gfedcba
    * cv2.BORDER_REPLICATE - Last element is replicated throughout, like this: aaaaaa|abcdefgh|hhhhhhh
    * cv2.BORDER_WRAP - Can’t explain, it will look like this : cdefgh|abcdefgh|abcdefg
* value - Color of border if border type is cv2.BORDER_CONSTANT

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
BLUE = [255,0,0]
img1 = cv2.imread('googlelogo.png')
replicate = cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_REPLICATE)
reflect = cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_REFLECT)
reflect101 = cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_REFLECT_101)
wrap = cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_WRAP)
constant= cv2.copyMakeBorder(img1,10,10,10,10,cv2.BORDER_CONSTANT,value=BLUE)
plt.subplot(231),plt.imshow(img1,'gray'),plt.title('ORIGINAL')
plt.subplot(232),plt.imshow(replicate,'gray'),plt.title('REPLICATE')
plt.subplot(233),plt.imshow(reflect,'gray'),plt.title('REFLECT')
plt.subplot(234),plt.imshow(reflect101,'gray'),plt.title('REFLECT_101')
plt.subplot(235),plt.imshow(wrap,'gray'),plt.title('WRAP')
plt.subplot(236),plt.imshow(constant,'gray'),plt.title('CONSTANT')
plt.show()

See the result above. (Image is displayed with matplotlib. So RED and BLUE planes will be interchanged).

___
# Arithmetic Operations on Images

### Image Addition

You can add two images by OpenCV function, cv2.add() or simply by numpy operation, res = img1 + img2.
Both images should be of same depth and type, or second image can just be a scalar value.

In [ ]:
import cv2
import numpy as np

# 500 x 300 sized images
img1 = cv2.imread('images/minionleft.jpg')
img2 = cv2.imread('images/minionright.jpg')

add = cv2.add(img1,img2)  # adds pixel values of first image to those of the second
cv2.imshow('add',add)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Image Blending
This is also image addition, but different weights are given to images so that it gives a feeling of blending or transparency.

In [ ]:
weighted = cv2.addWeighted(img1, 0.6, img2, 0.4, 0)  # affects opaqueness
cv2.imshow('weighted',weighted)

cv2.waitKey(0)
cv2.destroyAllWindows()

### Bitwise Operations
This includes bitwise AND, OR, NOT and XOR operations. They will be highly useful while extracting any part of
the image (as we will see in coming chapters), defining and working with non-rectangular ROI etc. Below we will see
an example on how to change a particular region of an image.

I want to put google logo above an image. If I add two images, it will change color. If I blend it, I get an transparent
effect. But I want it to be opaque. If it was a rectangular region, I could use ROI as we did in last chapter. But google
logo is a not a rectangular shape. So you can do it with bitwise operations as below:

In [ ]:
# Load two images
img1 = cv2.imread('images/minionright.jpg')
img2 = cv2.imread('images/googlelogo.png')

# I want to put logo on top-left corner, So I create a ROI
rows,cols,channels = img2.shape
roi = img1[0:rows, 0:cols ]

# Now create a mask of logo and create its inverse mask also
img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

# Now black-out the area of logo in ROI
img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)

# Take only region of logo from logo image.
img2_fg = cv2.bitwise_and(img2,img2,mask = mask)

# Put logo in ROI and modify the main image
dst = cv2.add(img1_bg,img2_fg)
img1[0:rows, 0:cols ] = dst
cv2.imshow('res',img1)
cv2.waitKey(0)
cv2.destroyAllWindows()

___